# Lab: Sinusoidal Signals

## Goals:

In this lab, you will explore properties of sinusoidal signals. Sinusoids are fundamental building blocks in signal processing. A thorough understanding of sinusoids is essential for mastering key concepts in signal analysis and processing.

In this lab, you will:

* practice your Python skills related to NumPy
* learn to synthesize sinusoidal signals from a given specification
* learn to measure the amplitude and phase from samples of a sinusoid
* explore the effect that delaying a sinusoid has on the phase of the signal
* explore the addition of sinusoids of the same frequency

## This notebook is incomplete

In this notebook, there are multiple places for you to fill in either code or text.

You should do that directly in this notebook. 

Once you have completed all your work in this notebook, rerun the entire notebook using "Kernel > Restart and Run All" from the menubar. 

Fix any errors, then remove this cell (your notebook is now complete), and submit.

In [ ]:
## import packages that we will need
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

## Part 1: Measuring the Phase of a Sinusoidal Signal

Our first goal is to devise a method for measuring the amplitude and phase of a sinusoidal signal from a set of $N$ samples taken at sampling rate $f_s$.

We will assume that the frequency $f$ of the sinusoid is known. Moreover, we will assume that this frequency satisfies $f = n \cdot \frac{f_s}{N}$ for some $n$ between 1 and $\frac{N}{2}$.

<p style="margin-bottom: 15px; padding: 4px 12px; background-color: #ffffcc; border-left: 6px solid #ffeb3b;">
For other frequencies, the method developed here gives only approximate measurements of amplitude and phase.
</p>

### Construct a sinusoidal signal

We have discussed before that a sinusoidal signal is synthesized in two steps:
1. create a time grid `tt`
2. use the time grid to compute samples

We want the samples of the sinusoid to be synthesized using the following parameters:
* sampling rate $f_s = 100$ Hz
* number of samples $N = 500$
  - this implies a duration of 5 s
* Amplitude $A=2$
* frequency $f = 6 \cdot \frac{f_s}{N} = 1.2$ Hz
* phase $\phi = -\frac{3\pi}{8}$

In [ ]:
# signal parameters
fs = 100
N = 500

A = 2
f = 6 * fs / N
phi = -3 * np.pi / 8

#### Task: Make a time grid

Use either `np.arange` or `np.linspace` to generate a time-grid `tt` according to the above specifications. The time grid should start at $t=0$.

In [ ]:
## make a time-grid
dur = N / fs

tt = FILL_ME_IN

## checks 
assert len(tt) == N, "the length of tt is not correct"
assert tt[-1] == (N-1) / fs, "the spacing between elements doesn't look right"

#### Task: Make signal samples

Using the time grid `tt` synthesize a sinusoidal signal with the above parameters. Store the resulting vector as `xx`.

Then, plot the signal you synthesized; make sure that the plot is properly labeled.

In [ ]:
## make samples xx
xx = FILL_ME_IN

## plot
FILL_ME_IN

plt.show()

### Breaking down the sinusoid

We can always write our sinusoidal signal as a sum of a cosine and a sine with zero phase.

To demonstrate that, we need the well-known trig identity
$$
    \cos(x+y) = \cos(x)\cos(y) - \sin(x)\sin(y).
$$

With this identity, we can write
$$
\begin{split}
x(t)  & = A\cos(2\pi ft + \phi) \\
 & = A\cos(\phi) \cos(2\pi ft) - A\sin(\phi) \sin(2\pi ft)
\end{split}
$$

Let's verify that this is true:
* synthesize sinusoids $x_I(t)=A\cos(\phi) \cos(2\pi ft)$ and $x_Q(t)=-A\cos(\phi) \cos(2\pi ft)$
* plot both sinusoids as well as the sum $x_I(t)+x_Q(t)$
* the sum should equal the original sinusoid above

In [ ]:
## plot three sinusoids
x_I = A*np.cos(phi) * np.cos(2*np.pi * f * tt)
x_Q = -A*np.sin(phi) * np.sin(2*np.pi * f * tt)

plt.plot(tt, x_I, label='$x_I(t)$')
plt.plot(tt, x_Q, label='$x_Q(t)$')
plt.plot(tt, x_I+x_Q, label='$x_I(t) + x_Q(t)$')

plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('x(t)')
plt.legend()

plt.show()

The green graph, $x_I(t) + x_Q(t)$, is identical to the plot of $x(t)$ above - as expected.

### Some Facts

For frequencies such that $f = \frac{f_s}{N}$, the following facts are true:
$$
\frac{2}{N} \sum_{n=0}^{N-1} \cos^2(2\pi f t[n]) = 1
$$
$$
\frac{2}{N} \sum_{n=0}^{N-1} \sin^2(2\pi f t[n]) = 1
$$
$$
\frac{2}{N} \sum_{n=0}^{N-1} \cos(2\pi f t[n])\sin(2\pi f t[n]) = 0
$$

Here $t[n]$ denotes the $n$-th sample of a time-grid with spacing $1/f_s$, i.e., $t[n]=\frac{n}{f_s}$. In other words, $t[n] is the $n$-th element of vector `tt`.

Let's check that these relationships are indeed correct.

In [ ]:
## verify the above facts
s_cos = np.cos(2* np.pi * f * tt)
s_sin = np.sin(2* np.pi * f * tt)

print('2/N * SUM cos^2 = ', 2/N * np.sum(s_cos**2))
print('2/N * SUM sin^2 = ', 2/N * np.sum(s_sin**2))
print('2/N * SUM cos * sin = ', 2/N * np.sum(s_cos * s_sin))

Well, that's about as close as you can get.

### Nearly There

With these facts and the split of the sinusoid into cosine and sine portions, we can compute the following for our sampled signal:
$$
A_c = \frac{2}{N} \sum_{n=0}^{N-1} x(t[n]) \cdot \cos(2\pi f t[n]) = A \cos(\phi)
$$
and
$$
A_c = -\frac{2}{N} \sum_{n=0}^{N-1} x(t[n]) \cdot \sin(2\pi f t[n]) = A \sin(\phi)
$$

Let's verify that as well:

In [ ]:
## verify the above expressions
print("2/N SUM x * cos = ", 2/N* np.sum(xx * s_cos), "A cos(phi) = ", A*np.cos(phi))
print("-2/N SUM x * sin = ", -2/N* np.sum(xx * s_sin), "A cos(phi) = ", A*np.sin(phi))

### Amplitude and Phase

From $A_c = A\cos(\phi)$ and $A_s = A\sin(\phi)$, we con now find $A$ and $\phi$ in terms of $A_c$ and $A_s$:

$$
A = \sqrt{A_c^2 + A_s^2}
$$
$$
\phi = \begin{cases}
\tan^{-1}(A_s/A_c) & \text{if $A_c > 0$} \\
\tan^{-1}(A_s/A_c) + \pi & \text{if $A_c < 0$} \\
\pi/2 & \text{if $A_c = 0$ and $A_s > 0$} \\
-\pi/2 & \text{if $A_c = 0$ and $A_s < 0$} 
\end{cases}
$$

This should look familar!

<p style="margin-bottom: 15px; padding: 4px 12px; background-color: #e7f3fe; border-left: 6px solid #2196F3;">
It is best to use the 2-argument inverse tangent function <tt>np.arctan2(y, x)</tt> to compute the phase.
</p>

### Pulling it all together

**Given:** a set of $N$ samples `xx` of a sinusoidal signal with frequency $f$

**Compute:**
1. $A_c$ and $A_s$ from the signal samples `xx` and zero-phase cosine and sine signals
$$
\begin{align}
A_c = & \frac{2}{N} \sum_{n=0}^{N-1} x(t[n]) \cos(2\pi f t[n]) \\
A_s = & \frac{2}{N} \sum_{n=0}^{N-1} x(t[n]) \sin(2\pi f t[n])
\end{align}
$$
2. Find the magnitude $A$ and phase $\phi$ from $A_c$ and $A_s$ as shown above.

Let's try this out on the signal `xx`. We should get the amplitude and phase that we specified ($A=2$ and $\phi=-\frac{3\pi}{8} = -0.375 \pi$).


In [ ]:
## Measure the amplitude and phase of xx
# 1. get Ac and As
Ac = 2/N* np.sum(xx * s_cos)
As = -2/N* np.sum(xx * s_sin)

# 2. turn into measured amplitude and phase
A_meas = np.sqrt(Ac**2 + As**2)
phi_meas = np.arctan2(As, Ac)

print("Measured amplitude: ", A_meas)
print(f"Measured phase {phi_meas / np.pi} * pi")

We now have a method to measure amplitude and phase that does not require us to read off values from a graph.

### This is so much easier with complex exponentials!

Instead of using cosine and sine functions in our procedure, we should use a complext exponential instead!

Specifically, 
* we can compute
$$
    X = \frac{2}{N} \sum_{n=0}^{N-1} x(t[n]) \exp(-j 2\pi f t[n])
$$
  - Note that $X = A_c + jA_s$.
* Thus, we need to find the polar representation of $X$, $X=Ae^{j\phi}$.

Much easier! Let's verify that this works

In [ ]:
## Measure amplitude and phase using a complex exponential
# 1. make a complex exponetial with frequency f
s_exp = np.exp(-2j*np.pi * f * tt)

# 2. compute X
X = 2/N * np.sum(xx * s_exp)

# 3. compute the polar representation
A_meas = np.abs(X)
phi_meas = np.angle(X)

print("Measured amplitude: ", A_meas)
print(f"Measured phase {phi_meas / np.pi} * pi")

Same result with a simpler computation.

Let's put our method to use.

## Part 2: The phase of a delayed sinusoid

We want to investigate what impact a simple signal processing operation has on a sinusoidal signal.

Specifically, we
* start with a sinusoid of unit amplitude and zero phase, sampled at rate $f_s$
* delay (i.e., shift right) by $n$ samples; this corresponds to a delay of $n/fs$ seconds
  - the function `np.roll()` does that
* measure the phase of the delayed sinusoid

For example, here is the effect that a delay of 20 samples has:

In [ ]:
## delay a sinusoid by five samples
# original sinusoid
ss = np.cos(2*np.pi * f *tt)

# delay 20 samples
dd = np.roll(ss, 20)

# measure phase of delayed signal
X = 2/N* np.sum(dd * s_exp)
phi_meas = np.angle(X)

print(f"Measured phase {phi_meas / np.pi:4.3f} * pi")

In [ ]:
## plot
plt.plot(tt, ss, label="original")
plt.plot(tt, dd, label="delayed")

plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('x(t)')
plt.legend()

plt.show()

### Task: Measure how the phase changes as a function of delay and explain

You are to repeat the above procedure for different delays in order to discover the relationship between delay and phase.

* Write a `for` loop that iterates over different delays 
  - use an integer `m` as your loop variable
  - `m` is the number of samples to delay by, i.e., the second input ro `np.roll`
  - it suffices to go to `m = 100`
* in each iteration of the loop:
    - store the delay `m/fs` in a list or vector named `tau`
    - delay the original signal by `m` samples
    - measure the phase of the delayed signal and store it in a list or vector named `phases`
* make a plot of phases versus delays


<p style="margin-bottom: 15px; padding: 4px 12px; background-color: #ffffcc; border-left: 6px solid #ffeb3b;">
In your plot, you may see a phase jump from -&pi; to &pi;. That is expected; the slope will be the same before and after the jump.
</p>

In [ ]:
## measure phase versus delay
# allocate vectors for results
M = 100
tau = np.zeros(M)
phases = np.zeros(M)

for m in range(M):
    # delay M samples
    FILL_ME_IN

In [ ]:
## plot phases versus delays
FILL_ME_IN

plt.show()

### Task: Measure the slope of the phase and explain

Your graph should show a (mostly) linear relationship between phase and delay.

* measure the slope of the line (segments) in your graph
  - you can use either of the straight line segments; they have the same slope
* Explain your result

**YOUR EXPLANATION GOES HERE**

## Summary

This lab covered a lot of ground:

* we used various NumPy functions to perform signal processing tasks, including synthesizing sinusoids, computing elemet-wise products of vectors and the sum of vector elements.
* we developed a method to measure the amplitude and phase of a sinusoid of known frequency
  - we will see this method again later in the course
* we measured how the phase changes when a sinusoidal signal is delayed.
  - this verified a result we discussed in class

  ### Deliverables

  Submit your complete copy of this notebook on Gradescope!
    * complete all the incomplete cells in this notebook
    * remove the cell that states that this notebook is incomplete.
    * **IMPORTANT** you must convert this notebook to PDF and upload a PDF copy - Do **NOT** upload the `.ipynb` notebook.
